In [58]:
from pathlib import Path

import httpx
from pymongo import MongoClient

DB_NAME = "validation_data"
COLLECTION_NAME = "api_response"

with open(Path.cwd() / ".env") as f:
    first_line, *_ = f.read().split("\n")
    idx = first_line.find("=")
    uri = first_line[idx + 1 :]


def get_db():
    client = MongoClient(uri)
    return client[DB_NAME]


db = get_db()
db

Database(MongoClient(host=['ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data')

In [59]:
collection = db[COLLECTION_NAME]
collection

Collection(Database(MongoClient(host=['ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data'), 'api_response')

In [60]:
async def get_prediction(lat, lon):
    async with httpx.AsyncClient() as client:
        url = f"http://localhost:8000/api/prediction?lat={lat}&lon={lon}&astro_twilight_type=previous"
        r = await client.get(url)
        return r.json()


prediction = await get_prediction(-30.2466, -70.7494)
prediction

{'brightness_mpsas': 19.7341,
 'astro_twilight': {'iso': '2023-12-30 01:24:01.832 UTC', 'type': 'previous'}}

In [61]:
collection.insert_one(prediction)